In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('2. match(AGN w BZCAT5).csv')

In [5]:
df.columns

Index(['Entry number', 'Fermi name', 'Counterpart name', 'Bzcat5 name',
       'Optical Class', 'RA (J2000.0)', 'Dec (J2000.0)', 'Redshift',
       'SED Class', 'Log Î½Sync. Z corr. (Hz)', 'Probability Bayesian',
       'Reliability LR RG', 'Reliability LR XG', 'Significance',
       'Flux 1-100 GeV (ph/cm2/s)', 'Spectral Index', 'Spectral Type',
       'Power Law Index', 'Radio flux (mJy)', 'X Flux (erg cm-2 s-1)',
       'USNO Vmag', 'SDSS Vmag', 'Seq', 'Name', 'RAh', 'RAm', 'RAs', 'DE-',
       'DEd', 'DEm', 'DEs', 'GLON', 'GLAT', 'z', 'u_z', 'Rmag', 'Class', 'FR',
       'F143', 'FX', 'FF', 'aro', 'RA_deg', 'DEC_deg', 'Separation'],
      dtype='object')